# Add new similarity aspects

This notebook illustates how a set of new similarity aspects can be added to the Atlas recommender configuration. A similarity aspect correponds to an instance of `EmbeddingModel` (stored in the model catalog) that is able to produce vectors describing particular data points (e.g. instances of `NeuronMorphology`) in some vector space, where similarity between two vectors represents similarity of the data points according to the given similarity aspect. For example, neurite features similarity of morphologies corresponds to a model that for each neuron morphology produces neurite feature vectors that can be used to establish a similarity score between two morphologies.

Typically, a record of the `RecommenderConfiguration` resources has the following shape

```
{
    "embeddingModel": MODEL_ID,
    "similarityView": MASTER_AGG_VIEW,
    "statisticsView": STATS_VIEW,
    "boostingView": AGG_VIEW_WITH_BOOSTING_FACTORS
}
```

where

- `MODEL_ID`, ID of the model representing the new similarity aspect;
- `MASTER_AGG_VIEW`, ID of the Master view, an aggregated ES view that collects a set of individual ES views from projects where data points reside. These individual views index embedding vectors produced by the above-mentioned model;
- `STATS_VIEW`, ID of the ES view with statistics on the Master view;
- `AGG_VIEW_WITH_BOOSTING_FACTORS`, ID of the aggregated ES view that collects a set of individual ES views with boosting factors from projects where data points reside.

Unfortunately, in Nexus, we cannot create an empty aggregated ES view, so the Master View (as well as Stats View and Agg Boosting View) needs to be created upon creation of the first ES view indexing a batch of embedding vectors. Therefore in this notebook, we create a new record that points only to the embedding model, but not the rest of the components.


Related JIRA tickets: 
* https://bbpteam.epfl.ch/project/issues/browse/DKE-718
* https://bbpteam.epfl.ch/project/issues/browse/DKE-715

Prerequisites:

- Models have been built and pushed to the model catalog
- Model meta-data has a `vectorDimension` field specifying the dimension of embedding vectors.

Steps:

1. Fetch the Recommender Configuration resource from the Atlas configuration project
2. Add new records to the `configuration` field pointing to the specified models.

In [17]:
import getpass
import os
import warnings

from kgforge.core import KnowledgeGraphForge

from bluegraph.downstream import EmbeddingPipeline
from bluegraph.core import GraphElementEmbedder

## User input

In [26]:
ENDPOINT = "https://staging.nexus.ocp.bbp.epfl.ch/v1"
DOWNLOAD_DIR = "./data"
TOKEN = getpass.getpass()

········


ID of the embedding models from the catalog

In [3]:
MODEL_IDS = [
    "https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/embedder_catalog/_/e2b953b9-6724-4278-a1e5-3472bd63e374",
    "https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/embedder_catalog/_/19700fed-a3af-4e7a-859d-ad223616ae2e",
    "https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/embedder_catalog/_/14d61701-c4fa-44ea-8139-0e0ed606b4ec"
]

Atlas configuration project

In [4]:
ATLAS_CONFIG_ORG = "dke"
ATLAS_CONFIG_PROJECT = "fake-atlas"

ID of the recommender configuration in the atlas config project.

In [5]:
ATLAS_RECOMMENDER_CONFIG = "https://bbp.epfl.ch/neurosciencegraph/data/415184d2-0ae5-4af0-89b7-8b00393b033f"

## Create sessions

Session for updating Atlas configs

In [27]:
forge_atlas = KnowledgeGraphForge(
    "https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
    endpoint=ENDPOINT,
    token=TOKEN, 
    bucket=f"{ATLAS_CONFIG_ORG}/{ATLAS_CONFIG_PROJECT}")

## Update Atlas recommender configuration 

In [28]:
config_resource = forge_atlas.retrieve(ATLAS_RECOMMENDER_CONFIG)

In [29]:
configuration_records = (
    [forge_atlas.as_json(el) for el in config_resource.configuration]
    if isinstance(config_resource.configuration, list)
    else [forge_atlas.as_json(config_resource.configuration)]
)
configured_modes = set([
    el["embeddingModel"] for el in configuration_records
])
for model_id in MODEL_IDS:
    if model_id in configured_modes:
        warnings.warn(
            f"A configuration record for mode '{model_id}' already exists, skipping",
            UserWarning)
    else:
        configuration_records.append({
            "embeddingModel": model_id,
            "similarityView": None,
            "statisticsView": None,
            "boostingView": None
        })
    
config_resource.configuration = forge_atlas.from_json(
    configuration_records)

/Users/oshurko/opt/anaconda3/envs/bg/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: A configuration record for mode 'https://staging.nexus.ocp.bbp.epfl.ch/v1/resources/dke/embedder_catalog/_/e2b953b9-6724-4278-a1e5-3472bd63e374' already exists, skipping
  del sys.path[0]


In [30]:
forge_atlas.update(config_resource)

<action> _update_one
<succeeded> True
